<a href="https://colab.research.google.com/github/kaniarasann/Langchain/blob/main/07.Tools_with_Orchestration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [195]:
#!pip install langchain langgraph langchain_groq pydantic feedparser beautifulsoup4 langchain-community langchain-exa

Imports

In [196]:
from pydantic import BaseModel,Field
from datetime import datetime
from dateutil import parser as dtparser
import feedparser
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from typing import List
import operator
from typing_extensions import Annotated
from langchain_exa import ExaSearchResults

Get top 5 news from the economic times this can be achieved with the help of RSS Feed : https://economictimes.indiatimes.com/markets/rssfeeds/1977021501.cms

In [197]:
url = 'https://economictimes.indiatimes.com/markets/rssfeeds/1977021501.cms'

In [198]:
# Class required for converting the Feed to pydantic class
class EconomicTime(BaseModel):
    title: str =''
    link: str = ''
    published: datetime =''
    content :str =''

In [199]:
#feed parse the Url add the nessecary package
feeds = feedparser.parse(url)

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()


In [200]:
#iterate through the first 5 feed
economic_times:list[EconomicTime] = []
for feed in feeds.entries[:5]:
    economic_times.append(EconomicTime(
        title = feed.title,
        link = feed.link,
        published = dtparser.parse(feed.published)

    ))


In [201]:
#get the content by using beautiful soup
from bs4 import BeautifulSoup
import requests

for news in economic_times:
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(news.link, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    article_div = soup.find("article")
    article_text = article_div.get_text(strip=True)
    news.content = article_text

In [202]:
economic_times[0].content

"ETMarkets.comThe Aditya Birla Group company incurred expenses of Rs 1,042 crore in the quarter under review versus Rs 1,313 crore in Q4FY25.India Cementson Saturday reported a consolidated net loss of Rs 131 crore for the June quarter compared to the profit of Rs 71 crore in the year-ago period. The company's revenue stood at Rs 1,025 crore, marginally lower than Rs 1,027 crore in the corresponding quarter of the previous financial year.The performance also marked a sharp downturn from the previous quarter, when the company had posted a profit of Rs 19 crore. Sequentially, revenue fell 14% from Rs 1,197 crore in Q4FY25.The Aditya Birla Group company incurred expenses of Rs 1,042 crore in the quarter under review versus Rs 1,313 crore in Q4FY25 and Rs 1,190 crore in Q1FY25. The expenses were made under the heads including 'Cost of Materials consumed', freight & forwarding expense and power & fuel, among other things.The company achieved domestic sales volume of 2.18 MnT, up 11.6% YoY. 

In [203]:
#create LLM
groq_key = userdata.get('groq_key')
llm = ChatGroq(api_key=groq_key, model="llama-3.3-70b-versatile",max_retries=5)
llm_reasoning = ChatGroq(api_key=groq_key, model="deepseek-r1-distill-llama-70b",max_retries=5)
search_tool = ExaSearchResults(exa_api_key= userdata.get('exa_api_key'))


In [204]:
class Stock(BaseModel):
    name: str = Field(description="Name of the stock")
    news_abstract: str = Field(description="Abstract of the news article related to the stock")

In [205]:
llm_stock_news = llm.with_structured_output(Stock)

In [206]:
economic_times

[EconomicTime(title='India Cements Q1 results: Co slips to Rs 131 crore loss, revenue flat YoY', link='https://economictimes.indiatimes.com/markets/stocks/earnings/india-cements-q1-results-co-slips-to-rs-131-crore-loss-revenue-flat-yoy/articleshow/122782761.cms', published=datetime.datetime(2025, 7, 19, 17, 31, 11, tzinfo=tzoffset(None, 19800)), content="ETMarkets.comThe Aditya Birla Group company incurred expenses of Rs 1,042 crore in the quarter under review versus Rs 1,313 crore in Q4FY25.India Cementson Saturday reported a consolidated net loss of Rs 131 crore for the June quarter compared to the profit of Rs 71 crore in the year-ago period. The company's revenue stood at Rs 1,025 crore, marginally lower than Rs 1,027 crore in the corresponding quarter of the previous financial year.The performance also marked a sharp downturn from the previous quarter, when the company had posted a profit of Rs 19 crore. Sequentially, revenue fell 14% from Rs 1,197 crore in Q4FY25.The Aditya Birla

In [207]:
stock_news:list[Stock] = []
def abstract_stock_new(news):
  prompt_stock = PromptTemplate.from_template("Following is the news:{news}. provide the company name and abstract news ").format(news=news.content)
  return llm_stock_news.invoke(prompt_stock)

for news in economic_times:
  stock_news.append(abstract_stock_new(news))


In [208]:
#Create State
class State(BaseModel):
    company_name: str = Field(description="Name of the company")
    news_abstract: str = Field(description="Abstract of the news article related to the company")
    queries:List[str] = []
    completed_queries: Annotated[List[str], operator.add] = []
    final_result: str = ''

class Query(BaseModel):
   query:str = ''

class Queries(BaseModel):
  queries:List[str] = []

In [209]:
#Worker State
class WorkerState(BaseModel):
    company_name: str = Field(description="Name of the company")
    completed_queries: Annotated[List[str], operator.add] = []

In [210]:
llm_queries = llm_reasoning.with_structured_output(Queries)
def orchestrator(state:State):
  promot_template = PromptTemplate.from_template("Your are Stock Market expert. Based on the following news list out the impacts in short / long:{news}").format(news=state.news)
  res = llm_queries.invoke(promot_template)
  return {'queries':res.queries}


In [211]:
promot_template = PromptTemplate.from_template("You are a stock market expert. Based on the following news, come up with the queries based on the news:{news}").format(news=stock_news[2].news_abstract)
res = llm_queries.invoke(promot_template)

In [212]:
for query in res.queries:
  print(query)

What factors contributed to the 33% surge in Central Bank of India's net profit in Q1 FY26?
How does this net profit compare to the previous quarter or the same quarter in the previous fiscal year?
What specific improvements in core income led to this growth?
What strategies has Central Bank of India implemented to reduce bad debts?
How does this performance impact the bank's future growth prospects?
What steps is Central Bank of India taking to sustain this level of profitability?
How does Central Bank of India's performance compare to its peers in the banking sector?


In [214]:
for queries in res.queries[:1]:
  # Perform a search query
  search_results = search_tool._run(
      query=queries,
      num_results=2,
      text_contents_options=True,
      highlights=True,
  )
  print(search_results)

Title: Central Bank of India Q1 Results: Net profit ... - The Economic Times
URL: https://m.economictimes.com/markets/stocks/earnings/central-bank-of-india-q1-results-net-profit-surges-33-to-rs-1169-crore/articleshow/122781433.cms
ID: https://m.economictimes.com/markets/stocks/earnings/central-bank-of-india-q1-results-net-profit-surges-33-to-rs-1169-crore/articleshow/122781433.cms
Score: None
Published Date: 2025-07-19T10:24:00.000Z
Author: PTI
Image: https://img.etimg.com/thumb/msid-122781463,width-1200,height-630,imgsize-49010,overlay-etmarkets/articleshow.jpg
Favicon: https://economictimes.indiatimes.com/icons/etfavicon.ico
Extras: None
Subpages: None
Text: ## Central Bank of India Q1 Results: Net profit surges 33% to Rs 1,169 crore

Central Bank of India Q1 Results: Net profit surges 33% to Rs 1,169 crore

PTILast Updated: Jul 19, 2025, 03:54:00 PM IST

Central Bank of India announced a significant 33 percent rise in net profit, reaching Rs 1,169 crore for the first quarter. This g